In [1]:
import sys 
sys.path.append('/host/d/Github')
import os
import torch
import numpy as np 
import nibabel as nb
import h5py
import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### extract image data

In [18]:
files = ff.find_all_target_files(['*.h5'],'/host/d/Data/minnie_EM/original_data/')
save_folder = '/host/d/Data/minnie_EM/nii'
for i in range(0,len(files)):
    file_name = files[i]
    print('file name is: ', file_name)
    # patient index
    patient_index = os.path.basename(file_name).split('.h5')[0][-3:]
    
    with h5py.File(file_name, 'r') as f:
        volume_data = f['volumes/image']
        
        volume_data_np = np.array(volume_data)
        shape = volume_data_np.shape
        if shape[0] < shape[1]:
            volume_data_np = np.transpose(volume_data_np, (1,2,0))
        assert np.max(volume_data_np) == 255
        assert np.min(volume_data_np) == 1 or np.min(volume_data_np) == 0

        volume_data_np = Data_processing.normalize_image(volume_data_np, image_max = np.max(volume_data_np), image_min = np.min(volume_data_np), final_max = 1.0, final_min = 0.0, normalize_factor = 'equation')
        print('data shape before removing empty slices: ', volume_data_np.shape)
       
        # remove the slices that have all zero pixels
        total_slices = volume_data_np.shape[2]
        valid_slices = []
        for slice_index in range(total_slices):
            slice_data = volume_data_np[:,:,slice_index]
            if np.sum(slice_data) > 0:
                valid_slices.append(slice_index)
        volume_data_np = volume_data_np[:,:,valid_slices]
        print('after removing empty slices, the new shape is: ', volume_data_np.shape)

        ff.make_folder([os.path.join(save_folder, 'patient_'+patient_index)])
        nb.save(nb.Nifti1Image(volume_data_np, affine = np.eye(4)), os.path.join(save_folder, 'patient_'+patient_index, 'img.nii.gz'))
    

file name is:  /host/d/Data/minnie_EM/original_data/minnie_vol001.h5
data shape before removing empty slices:  (768, 769, 76)
after removing empty slices, the new shape is:  (768, 769, 75)
file name is:  /host/d/Data/minnie_EM/original_data/minnie_vol002.h5
data shape before removing empty slices:  (768, 768, 76)
after removing empty slices, the new shape is:  (768, 768, 75)
file name is:  /host/d/Data/minnie_EM/original_data/minnie_vol003.h5
data shape before removing empty slices:  (768, 768, 76)
after removing empty slices, the new shape is:  (768, 768, 75)
file name is:  /host/d/Data/minnie_EM/original_data/minnie_vol004.h5
data shape before removing empty slices:  (768, 768, 76)
after removing empty slices, the new shape is:  (768, 768, 75)
file name is:  /host/d/Data/minnie_EM/original_data/minnie_vol005.h5
data shape before removing empty slices:  (768, 768, 76)
after removing empty slices, the new shape is:  (768, 768, 75)
file name is:  /host/d/Data/minnie_EM/original_data/min

### add noise

In [5]:
def compute_snr(clean, noisy, eps=1e-12):
    """
    clean, noisy: np.ndarray, 已经在 [0,1] 区间
    返回: SNR (dB)
    """
    clean = clean.astype(np.float32)
    noisy = noisy.astype(np.float32)
    noise = noisy - clean

    signal_power = np.mean(clean**2)
    noise_power  = np.mean(noise**2) + eps

    snr = 10.0 * np.log10(signal_power / noise_power)
    return snr

In [19]:
def add_noise_EMslice(x, seed, sigma1_range = [0.45, 0.55], sigma2_range = [1.8, 2.2], sigma_target_range = [0.2,0.4]):
    
    rng = np.random.default_rng(seed)
    eps = rng.standard_normal(size = x.shape).astype(np.float32)

    seed_sigma = seed + 1
    np.random.seed(seed_sigma)
    sigma1 = np.random.uniform(sigma1_range[0], sigma1_range[1])
    sigma2 = np.random.uniform(sigma2_range[0], sigma2_range[1])
    low1 = gaussian_filter(eps, sigma=sigma1)   # H_sigma1(eps)
    low2 = gaussian_filter(eps, sigma=sigma2)   # H_sigma2(eps)
    z = low1 - low2                             # z = (H_sigma1 - H_sigma2)(eps)

    sigma_target = np.random.uniform(sigma_target_range[0], sigma_target_range[1])
    alpha = sigma_target / (z.std() + 1e-8)

    y = np.copy(x) + alpha * z
    y = np.clip(y, 0.0, 1.0)
    return y, [z, sigma1, sigma2, sigma_target, alpha]
   

In [31]:
from scipy.ndimage import gaussian_filter
save_folder = '/host/d/Data/minnie_EM/simulation_v1'
ff.make_folder([save_folder])

files = ff.find_all_target_files(['*/img.nii.gz'],'/host/d/Data/minnie_EM/nii')

simulation_num = 2

for file_index in range(0,len(files)):
    patient_id = os.path.basename(os.path.dirname(files[file_index]))


    file = files[file_index]
    clean_img = nb.load(file).get_fdata()
    print('patient id: ', patient_id)

    for simulation_index in range(1, simulation_num+1):

        # if os.path.isfile(os.path.join(save_folder, patient_id, 'recon_'+ str(simulation_index), 'img.nii.gz')):
        #     print('file exists, skiping... ')
        #     continue
        
        noisy_img = np.zeros_like(clean_img)

        for slice_index in range(0, clean_img.shape[-1]):
            x = clean_img[:,:,slice_index]

            seed = file_index * 10000 + simulation_index * 1000 + slice_index * 100

            y, _ = add_noise_EMslice(x, seed, sigma1_range = [0.50, 0.65], sigma2_range = [2,3], sigma_target_range = [0.3,0.4])

            noisy_img[:,:,slice_index] = y

        # calculate snr
        snr_db = compute_snr(clean=clean_img, noisy=noisy_img)
        # save
        # print('simulation index: ', simulation_index, ' SNR (dB): ', snr_db)
        save_path = os.path.join(save_folder, patient_id, 'recon_'+ str(simulation_index), 'img.nii.gz')
        ff.make_folder([os.path.join(save_folder, patient_id), os.path.join(save_folder, patient_id, 'recon_'+ str(simulation_index))])
        nb.save(nb.Nifti1Image(noisy_img, affine = np.eye(4)), save_path)

patient id:  patient_001
patient id:  patient_002
patient id:  patient_003
patient id:  patient_004
patient id:  patient_005
patient id:  patient_006
patient id:  patient_007
patient id:  patient_008
patient id:  patient_009
patient id:  patient_010
patient id:  patient_011
patient id:  patient_012
patient id:  patient_013
patient id:  patient_014
patient id:  patient_015
patient id:  patient_016
patient id:  patient_018
patient id:  patient_019
patient id:  patient_020
patient id:  patient_021
patient id:  patient_022
patient id:  patient_023
patient id:  patient_024
patient id:  patient_025
patient id:  patient_026
patient id:  patient_027
patient id:  patient_028


In [29]:
files = ff.find_all_target_files(['patient_001/recon_*/img.nii.gz'],'/host/d/Data/minnie_EM/simulation_v1')
for i in range(0,len(files)):
    file = files[i]
    img = nb.load(file).get_fdata()
    if i == 0:
        final_collection = img[..., np.newaxis]
    else:
        final_collection = np.concatenate((final_collection, img[..., np.newaxis]), axis = -1)
final_image = np.mean(final_collection, axis = -1)
nb.save(nb.Nifti1Image(final_image, affine = np.eye(4)), '/host/d/Data/minnie_EM/simulation_v1/patient_001/ensemble_mean_img.nii.gz')